In [2]:
import tensorflow.compat.v1 as tf #使用tensorflow 1
from PIL import Image
import os
tf.disable_v2_behavior()

In [4]:
#功能：制作tfrecord数据集
#参数：数据集存储路径，包含所有图片文件夹的文件夹路径
def write_tfRecord(tfRecordPath, allFoldersPath):
    writer = tf.python_io.TFRecordWriter(tfRecordPath)  # 新建一个writer
    folderPathList = os.listdir(allFoldersPath)
    labelNumber = len(folderPathList)
    labelFlag = 0
    for folderName in folderPathList:
        folderPath = allFoldersPath+folderName+'/'
        imagePathList = os.listdir(folderPath)
        for imageName in imagePathList:
            img_path = folderPath+imageName
            preimg = Image.open(img_path)
            img = preimg.resize((64, 64))#将图像转为64*64
            img_raw = img.tobytes() # 转化为二进制文件
            labels = [0] * labelNumber
            labels[int(labelFlag)] = 1
            # 用tf.train.Example的协议存储训练数据，训练数据的特征用键值对的形式表示
            example = tf.train.Example(features=tf.train.Features(feature={
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
                    }))     # 把每张图片和标签封装到example中
            writer.write(example.SerializeToString())   # 将example序列化(把数据序列化成字符串存储)
        labelFlag += 1
    writer.close() # 关闭writer
    print('Finish')
    
write_tfRecord('C:/Users/lenovo/Desktop/jupyter_notebook/BigChuang/dataSet/test.tfrecords','F:/deepLearning/trainingData2/NewYorkHouse/')

Finish


In [3]:
#生成pb模型
import tensorflow as tf #使用tensorflow 1
#from PIL import Image
import os
from tensorflow.python.framework import graph_util

def read_tfRecord(tfRecord_path):
    filename_queue = tf.train.string_input_producer([tfRecord_path], shuffle=True)
    reader = tf.TFRecordReader() 
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                        'label': tf.FixedLenFeature([4], tf.int64),
                                        'img_raw': tf.FixedLenFeature([], tf.string)
                                        })  
    label = features['label']
    img = features['img_raw']
    img = tf.decode_raw(img, tf.uint8)
    img = tf.reshape(img, [64, 64, 3])
    img = tf.cast(img, tf.float32) * (1. / 255)-0.5  
    label = tf.cast(label, tf.int32)
    return img, label


def get_tfrecord(num, tfRecord_path):
    img, label = read_tfRecord(tfRecord_path)
    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                    batch_size = num,
                                                    capacity = 2000,
                                                    min_after_dequeue = 0)
    return img_batch, label_batch

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),
                                 tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                     tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,
                                         ys:v_ys})
    return result
    
def weight_variable(shape):
    inital = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(inital,name='w')
    
def bias_variable(shape):
    inital = tf.constant(0.1,shape=shape)
    return tf.Variable(inital,name='b')
    
def conv2d(x,W):
    #对输入的x用W做卷积操作
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],
                       padding='SAME')
                       
def max_pool_2x2(x):
    #对输入的x做pooling，使图像尺寸减半
    return tf.nn.max_pool(x,ksize=[1,2,2,1],
                         strides=[1,2,2,1],
                         padding='SAME')
                         
xs = tf.placeholder(tf.float32,[None,64,64,3],name='xs')#图片格式
ys = tf.placeholder(tf.float32,[None,4])#标签数目
x_image = tf.reshape(xs,[-1,64,64,3])

with tf.variable_scope("conv1"):
    W_conv1 = weight_variable([5,5,3,32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1) #64 64 32
    h_pool1 = max_pool_2x2(h_conv1)#32 32 32

with tf.variable_scope("conv2"):
    W_conv2 = weight_variable([5,5,32,64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)#32 32 64
    h_pool2 = max_pool_2x2(h_conv2)#16 16 64

with tf.variable_scope("fc1"):
    W_fc1 = weight_variable([16*16*64,1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2,[-1,16*16*64])#20 16*16*64
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)#20 1024

with tf.variable_scope("fc2"):
    W_fc2 = weight_variable([1024,4])
    b_fc2 = bias_variable([4]) #标签数
    
prediction = tf.nn.softmax(tf.matmul(h_fc1,W_fc2)+b_fc2,name='prediction') #20 2

cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),
                                             reduction_indices=[1]))#loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)#反向传播

sess = tf.Session()
sess.run(tf.initialize_all_variables())

path = 'F:/deepLearning/trainingData2/test.tfrecords'

train_xs,train_ys = get_tfrecord(98,path)
tf.train.start_queue_runners(sess=sess)#将数据从内存获取进行计算

for i in range(50):    
    #用上述获取的200张图训练350次，每10次用上述的1000张图检验一次准确率
    train_images,train_labels = sess.run([train_xs,train_ys])
    #将tensor转换为ndarray
    sess.run(train_step,feed_dict={xs:train_images,ys:train_labels})
    
constant_graph = graph_util.convert_variables_to_constants(sess,
                                                        sess.graph_def,
                                                        ['prediction'])

with tf.gfile.FastGFile('C:/Users/lenovo/Desktop/jupyter_notebook/BigChuang/dataSet/model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())
print('Finish')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input